<a href="https://colab.research.google.com/github/ntrajic/AgenticRoomReservation/blob/main/ADK_ToolKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 The Ultimate ADK Toolkit: A Developer's Guide to Tool Integration 🚀

Welcome, Agent Architect! This notebook is your definitive guide to giving AI agents superpowers through **tool integration**. An agent's true power isn't just its language model; it's its ability to connect to and interact with the outside world. We will explore the entire spectrum of tool integration patterns available in the Google Agent Development Kit (ADK).

By the end of this adventure, you will master how to:

- ✅ **Use Built-in & Custom Function Tools**: The foundational patterns for giving an agent new skills.
- ✅ **Generate Tools from Specifications**: Automatically create toolsets from **OpenAPI** specs and connect to enterprise APIs in **Google Cloud API Hub**.
- ✅ **Connect to Live Tool Servers**: Use **MCP** to have your agent dynamically discover tools hosted anywhere on the web.
- ✅ **Integrate with Other Frameworks**: Seamlessly use tools from the **LangChain** ecosystem directly within your ADK agent.
- ✅ **Share State Between Tools**: Use `ToolContext` to enable complex, multi-step workflows where tools can pass information to each other.
- ✅ **Build Multi-Agent Systems**: Create sophisticated systems where a primary "orchestrator" agent can delegate tasks to specialist agents.

Let's dive into the toolkit!

---
## Author

Hi, I'm Qingyue (Annie) Wang, a Developer Advocate and AI Engineer at **Google**. I'm passionate about helping developers build amazing things with AI and cloud technologies.

If you have questions about this notebook, feel free to reach out on [LinkedIn](https://www.linkedin.com/in/qingyuewang/) or [X (formerly Twitter)](https://twitter.com/qingyuewang).

ADK Agents Tutorial: https://codelabs.developers.google.com/onramp/instructions#1

```
 (\__/)
 (•ㅅ•)
 /づ  📚      Enjoy learning about AI Agents!
```

---
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

You will need a **Google AI API Key** to run this notebook.

👉 **Get Your Key HERE**: [https://codelabs.developers.google.com/onramp/instructions#1](https://codelabs.developers.google.com/onramp/instructions#1)

*Note: The LangChain integration in Part 4 requires an additional API key from a service like Tavily, which has a free tier.*

-----------------------------------------------------------------------------
```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\      /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )   ( o_o )
```

---
## Part 0: Setup & Authentication 🔑

Let's install all necessary libraries and configure your API key. This single setup will prepare you for every example in the notebook.

In [ ]:
# Install all the packages we'll need for this entire tutorial
# This includes the ADK, Google's AI library, and libraries for specific integrations
!pip install google-adk google-generativeai mcp requests nest-asyncio langchain-community tavily-python wikipedia -q

# --- Import all necessary libraries ---
import asyncio
import os
import json
import requests
import traceback
from getpass import getpass
from IPython.display import display, Markdown

# ADK and Tool imports
from google.adk.agents import Agent
from google.adk.tools import google_search, ToolContext
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.openapi_tool import OpenAPIToolset
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams
from google.adk.tools.langchain_tool import LangchainTool
# Note: APIHubToolset is shown for conceptual purposes
# from google.adk.tools.apihub_tool import APIHubToolset

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session

# Google AI imports
import google.generativeai as genai
from google.genai.types import Content, Part

# LangChain imports for Part 4
from langchain_community.tools import TavilySearchResults

# A helper to run async ADK code in Colab/Jupyter
import nest_asyncio
nest_asyncio.apply()

print("✅ All libraries are installed and ready to go!")

✅ All libraries are installed and ready to go!


In [ ]:
google_api_key = getpass('Enter your Google API Key: ')
genai.configure(api_key=google_api_key)
os.environ['GOOGLE_API_KEY'] = google_api_key
print("✅ Google API Key configured.")

Enter your Google API Key: ··········
✅ Google API Key configured.


### A Helper Function to Run Our Agents
To keep our code clean, we'll define a single helper function to manage running our agents.

In [ ]:
# Initialize a session service to manage conversations
session_service = InMemorySessionService()
my_user_id = "adk_toolkit_user"

async def run_agent_query(agent: Agent, query: str):
    """A reusable function to run a query against any agent."""
    session = await session_service.create_session(app_name=agent.name, user_id=my_user_id)
    print(f"\n🚀 Running query for agent: '{agent.name}'...")

    runner = Runner(agent=agent, session_service=session_service, app_name=agent.name)
    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=my_user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # To see the agent's full thought process, uncomment the line below
            # print(f"EVENT: {event}")
            if event.is_final_response() and event.content.parts:
                final_response += event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"
        print("\n🔍 Full traceback:")
        print(traceback.format_exc())

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")
    return final_response

print("✅ Agent query helper function defined.")

✅ Agent query helper function defined.


---

## Part 1: The Basics - Direct Tooling

These are the most fundamental ways to give an agent new skills.

### 1.1 Built-in Tools
The ADK comes with pre-packaged tools. `Google Search` is the perfect example, giving your agent immediate access to real-time information.

In [ ]:
# Define an agent and give it the built-in Google Search tool
search_agent = Agent(
    name="search_agent",
    model="gemini-2.5-flash",
    instruction="You are a helpful search assistant. Answer the user's question using Google Search.",
    tools=[google_search]
)

# Run the agent
await run_agent_query(search_agent, "What is the latest news about the Artemis program?")


🚀 Running query for agent: 'search_agent'...

--------------------------------------------------
✅ Final Response:


The Artemis program, led by NASA, continues to advance with several key developments and updated timelines for its ambitious lunar exploration missions. The program aims to establish a long-term human presence on the Moon and prepare for future human missions to Mars.

Here's a summary of the latest news:

**Mission Schedule Updates:**
*   **Artemis II:** The first crewed Artemis mission, a lunar flyby, is now targeting a launch in **early 2026**, potentially as early as February 2026. This is an acceleration from a previously announced April 2026 target, though it followed earlier delays from September 2025 due to issues with the Artemis I heat shield and life support system analysis. The European Service Module for this mission was handed over to NASA in 2023, and testing on the Orion module is underway. The SLS for Artemis II is fully stacked on Mobile Launcher 1, awaiting the Orion spacecraft and its ascent abort motor fairing.
*   **Artemis III:** This mission, planned to be the first American crewed lunar landing since 1972, is now scheduled for **mid-2027**. It was previously delayed from September 2026, also due to the Artemis I heat shield concerns. Artemis III will utilize Blue Origin's Blue Moon lander to transport astronauts to the Moon's surface.
*   **Artemis IV:** The mission to dock with the Lunar Gateway space station, remains on track for **late 2028**.
*   **Artemis V:** This mission, which will deliver four astronauts to the Lunar Gateway and be the third lunar landing, is planned for **early 2030**. It will also deliver two new elements to the Gateway space station and be the first lunar landing since Apollo 17 to use an unpressurized lunar rover.
*   **Artemis VI:** Expected in **early 2031**, this mission will integrate the Crew and Science Airlock with the Lunar Gateway station.
*   **Artemis VII:** Planned for **early 2032**, this mission aims to deliver the Habitable Mobility Platform (Lunar Cruiser) to the lunar surface.
*   After Artemis VI, NASA plans yearly landings on the Moon.

**Recent Developments and Activities:**
*   In April 2024, three companies won NASA contracts to develop Artemis moon rover designs, competing for a single contract for actual development after a year to perfect their designs.
*   Commercial moon landers are actively involved, with "Blue Moon" from Firefly and "Resilience" from ispace being carried into space by a SpaceX rocket in January, part of an ongoing push for private-sector lunar exploration experience.
*   A ship-to-ship propellant transfer demonstration is expected in 2025 to further prove out this capability.
*   In June 2024, the first integrated test for Artemis III was conducted, including next-generation spacesuits by Axiom Space and the airlock module of Starship HLS.
*   Teams at NASA's Kennedy Space Center practiced Artemis mission emergency escape procedures in August 2024.
*   The Orion spacecraft for Artemis II was delivered to NASA in May 2025 for prelaunch preparations.
*   The liquid hydrogen tank for the Artemis III Space Launch System rocket was moved into its final assembly area in April 2025.
*   The habitat module for the Moon-orbiting Gateway space station arrived in the US.
*   Norway became the 55th nation to sign the NASA Artemis Accords for peaceful space exploration.
*   There was an anomaly during a static fire test of a solid rocket engine for NASA's Space Launch System rocket in June 2025, where a nozzle blew off.

**Future Outlook:**
*   NASA is looking for ways to enable an earlier launch for Artemis II if possible.
*   NASA has asked SpaceX and Blue Origin to begin applying their landing system development knowledge towards future variations that could deliver large cargo on later missions.
*   The proposed NASA budget for Fiscal Year 2026 aims to end the SLS and Orion programs after the Artemis III lunar landing, creating uncertainty for the program's future beyond that mission.

--------------------------------------------------



'The Artemis program, led by NASA, continues to advance with several key developments and updated timelines for its ambitious lunar exploration missions. The program aims to establish a long-term human presence on the Moon and prepare for future human missions to Mars.\n\nHere\'s a summary of the latest news:\n\n**Mission Schedule Updates:**\n*   **Artemis II:** The first crewed Artemis mission, a lunar flyby, is now targeting a launch in **early 2026**, potentially as early as February 2026. This is an acceleration from a previously announced April 2026 target, though it followed earlier delays from September 2025 due to issues with the Artemis I heat shield and life support system analysis. The European Service Module for this mission was handed over to NASA in 2023, and testing on the Orion module is underway. The SLS for Artemis II is fully stacked on Mobile Launcher 1, awaiting the Orion spacecraft and its ascent abort motor fairing.\n*   **Artemis III:** This mission, planned to 

### 1.2 Custom Function Tools
This is the most common pattern: wrapping your own Python function into a tool. The function's **docstring** is crucial, as it tells the LLM what the tool does and when to use it. Here, we'll create a tool to fetch live weather data from the public U.S. National Weather Service API.

In [ ]:
# --- Tool Definition: A function that calls a live public API ---
# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"\n🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')\n")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---
weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

# --- Let's test the Weather-Aware Planner ---
await run_agent_query(weather_agent, "I want to go hiking near Lake Tahoe tomorrow, what's the weather like?")

🌦️ Agent 'weather_aware_planner' is created and can now call a live weather API!

🚀 Running query for agent: 'weather_aware_planner'...



🛠️ TOOL CALLED: get_live_weather_forecast(location='Lake Tahoe')




--------------------------------------------------
✅ Final Response:


The weather near Lake Tahoe tomorrow will be mostly clear, with a low around 52°F and a west wind around 5 mph. It sounds like great weather for a hike!

--------------------------------------------------



'The weather near Lake Tahoe tomorrow will be mostly clear, with a low around 52°F and a west wind around 5 mph. It sounds like great weather for a hike!'

---
## Part 2: Specification-Driven Tools

Instead of writing function by function, you can provide the agent with a formal specification, and it will generate the necessary tools automatically.

### 2.1 From an OpenAPI Spec (`OpenAPIToolset`)
If you have an existing API with an OpenAPI specification, you can instantly turn it into a toolset for your agent. This is incredibly powerful for integrating with existing microservices.

In [ ]:
# A simple OpenAPI 3.0 spec for a Pet Store as a string
pet_store_spec = """
openapi: 3.0.0
info:
  title: Simple Pet Store API
  version: 1.0.0
servers:
  - url: https://petstore.swagger.io/v2
paths:
  /pet/{petId}:
    get:
      summary: Find pet by ID
      operationId: getPetById
      parameters:
        - name: petId
          in: path
          required: true
          schema:
            type: integer
            format: int64
      responses:
        '200':
          description: successful operation
          content:
            application/json:
              schema:
                type: object
                properties:
                  id:
                    type: integer
                  name:
                    type: string
                  status:
                    type: string
"""

# 1. Create the toolset from the OpenAPI spec string
pet_store_toolset = OpenAPIToolset(
    spec_str=pet_store_spec, spec_str_type='yaml'
)

# 2. Create an agent that uses this toolset
pet_store_agent = Agent(
    name="pet_store_agent",
    model="gemini-2.5-flash",
    instruction="You are a Pet Store assistant. Use your tools to find information about pets.",
    tools=[pet_store_toolset]
)

# 3. Run the agent
# The agent will see the `getPetById` tool and know how to call the API.
await run_agent_query(pet_store_agent, "Can you find the pet with ID 5?")


🚀 Running query for agent: 'pet_store_agent'...



--------------------------------------------------
✅ Final Response:


Here is the pet with ID 5: doggie.

--------------------------------------------------



'Here is the pet with ID 5: doggie.'

### 2.2 From a Public Server (`MCPToolset`)
**MCP** is a standard that lets agents discover tools from a remote server. We can use `MCPToolset` to connect to a public server hosting tools for the MDN Web Docs.

In [ ]:
# The URL for the public server hosting MDN documentation tools
MCP_SERVER_URL = "https://gitmcp.io/mdn/content"

# This is an async function because MCPToolset needs to connect to the server
async def create_mdn_agent():
    mcp_toolset = MCPToolset(
        connection_params=SseServerParams(url=MCP_SERVER_URL)
    )
    return Agent(
        name="mdn_docs_assistant",
        model="gemini-2.5-flash",
        tools=[mcp_toolset],
        instruction="You are a web dev expert with access to MDN docs. Use your tools to answer questions."
    )

# Create and run the agent
mdn_agent = await create_mdn_agent()
await run_agent_query(mdn_agent, "What is the CSS `box-sizing` property?")


🚀 Running query for agent: 'mdn_docs_assistant'...


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:87: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(



--------------------------------------------------
✅ Final Response:


The CSS `box-sizing` property defines how the total width and height of an element are calculated.

By default, with `box-sizing: content-box` (the initial CSS standard behavior), the `width` and `height` you set for an element apply only to its content box. Any padding or border added to the element will increase its total rendered width and height beyond the specified values.

When `box-sizing` is set to `border-box`, the `width` and `height` properties include the content, padding, and border. This means that the total rendered width and height of the element will be exactly what you specify, and the content area will shrink to accommodate any added padding and border. This often makes sizing elements much easier in layout design.

For example:
*   If an element has `width: 100px; padding: 10px; border: 5px solid black;`
    *   With `box-sizing: content-box;`, the element's total width will be 100px (content) + 2\*10px (padding) + 2\*5px (border) = 130px.
    *   With `box-sizing: border-box;`, the element's total width will be 100px. The content area will be 100px - 2\*10px (padding) - 2\*5px (border) = 70px.

--------------------------------------------------



"The CSS `box-sizing` property defines how the total width and height of an element are calculated.\n\nBy default, with `box-sizing: content-box` (the initial CSS standard behavior), the `width` and `height` you set for an element apply only to its content box. Any padding or border added to the element will increase its total rendered width and height beyond the specified values.\n\nWhen `box-sizing` is set to `border-box`, the `width` and `height` properties include the content, padding, and border. This means that the total rendered width and height of the element will be exactly what you specify, and the content area will shrink to accommodate any added padding and border. This often makes sizing elements much easier in layout design.\n\nFor example:\n*   If an element has `width: 100px; padding: 10px; border: 5px solid black;`\n    *   With `box-sizing: content-box;`, the element's total width will be 100px (content) + 2\\*10px (padding) + 2\\*5px (border) = 130px.\n    *   With `

---
## Part 3: Interoperability

You don't have to build everything from scratch. The ADK is designed to work with other popular AI frameworks.

### 3.1 Using LangChain Tools (`LangchainTool`)
If you have existing tools built with LangChain, you can wrap them using `LangchainTool` and use them directly in your ADK agent. Here, we'll wrap LangChain's `WikipediaAPIWrapper` tool.

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from google.adk.tools.langchain_tool import LangchainTool

# 1. Instantiate the original LangChain tool
# This tool queries the public Wikipedia API.
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 2. Wrap it for ADK using LangchainTool
adk_wrapped_wiki_tool = LangchainTool(tool=wikipedia_tool)

# 3. Use the wrapped tool in your ADK agent
wiki_agent = Agent(
    name="wiki_research_agent",
    model="gemini-2.5-flash",
    instruction="You are a research assistant. Use the Wikipedia tool to answer the user's question.",
    tools=[adk_wrapped_wiki_tool]
)

# 4. Run the agent
await run_agent_query(wiki_agent, "What is the history of the Slinky toy?")


🚀 Running query for agent: 'wiki_research_agent'...



--------------------------------------------------
✅ Final Response:


The Slinky toy was invented and developed by American naval engineer Richard T. James in 1943. It was successfully demonstrated at Gimbels department store in Philadelphia on November 27, 1945.

--------------------------------------------------



'The Slinky toy was invented and developed by American naval engineer Richard T. James in 1943. It was successfully demonstrated at Gimbels department store in Philadelphia on November 27, 1945.'

---
## Part 4: Advanced Composition

Now let's combine these concepts to build more sophisticated systems.

### 4.1 Sharing State Between Tools (`ToolContext`)
What if one tool needs information gathered by another? `ToolContext` is a special object that can be passed to your tool functions, allowing them to read and write to a shared state dictionary within a single conversational turn.

In [ ]:
def set_user_preference(preference: str, value: str, tool_context: ToolContext):
    """Saves a user's preference for this session.

    Args:
        preference: The name of the preference (e.g., 'theme').
        value: The value of the preference (e.g., 'dark').
    """
    # The state is a simple dictionary unique to this turn
    tool_context.state[preference] = value
    print(f"\n🛠️ TOOL CALLED: Set preference '{preference}' to '{value}'\n")
    return {"status": "success", "message": f"Preference saved."}

def get_user_preference(preference: str, tool_context: ToolContext):
    """Gets a previously saved user preference.

    Args:
        preference: The name of the preference to retrieve.
    """
    value = tool_context.state.get(preference, "not set")
    print(f"\n🛠️ TOOL CALLED: Retrieved preference '{preference}', value is '{value}'\n")
    return {"preference": preference, "value": value}


stateful_agent = Agent(
    name="stateful_agent",
    model="gemini-2.5-flash",
    instruction="First, save the user's preference. Then, retrieve it and confirm it back to them.",
    tools=[set_user_preference, get_user_preference]
)

await run_agent_query(stateful_agent, "Please set my theme preference to 'dark mode'.")


🚀 Running query for agent: 'stateful_agent'...



🛠️ TOOL CALLED: Set preference 'theme' to 'dark mode'


--------------------------------------------------
✅ Final Response:


OK. I've set your theme preference to 'dark mode'.


--------------------------------------------------



"OK. I've set your theme preference to 'dark mode'.\n"

### 4.2 Agents as Tools (`AgentTool`)
The ultimate composition pattern: using an entire agent as a tool for another agent. This lets you create a primary "orchestrator" that delegates complex tasks to a team of specialists.

This example is the perfect demonstration of `ToolContext` in action. The orchestrator needs to perform a two-step process:
1.  Call the `db_agent` to get a list of hotels.
2.  Call the `concierge_agent` with the hotel data to get a recommendation.

The `ToolContext` acts as a temporary "clipboard" or "briefcase" for the turn. The first tool (`call_db_agent`) places the hotel data into `tool_context.state`, and the second tool (`call_concierge_agent`) retrieves it.


```
                                 +-----------------------------------------------------------+
                                 |                      🧭 TripDataConcierge                 |
                                 |                        (Orchestrator)                     |
                                 +-----------------------------------------------------------+
                                            /                         \
                                           /                           \
               +----------------------------------+      +--------------------------------------+
               | 🔧 Tool: call_db_agent           |      | 🔧 Tool: call_concierge_agent          |
               | Writes data to `tool_context`    |      | Reads data from `tool_context`         |
               | Calls: 📦 db_agent (for data)    |      | Calls: 🤵 concierge_agent (for advice) |
               +----------------------------------+      +--------------------------------------+
                                                                              |
                                                                              ▼
                                                           +------------------------------------+
                                                           | 🤵 concierge_agent                 |
                                                           | Tools: [ 🍽️ food_critic_agent ]    |
                                                           +------------------------------------+
                                                                              |
                                                                              ▼
                                                           +------------------------------------+
                                                           | 🍽️ food_critic_agent               |
                                                           | (Gives witty recommendations)      |
                                                           +------------------------------------+
```

In [ ]:
# A mock database agent. In a real app, this might be a NL-to-SQL agent.
db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(question: str, tool_context: ToolContext):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("\n--- TOOL CALL: Delegating to db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )

    # *** WRITING TO THE CONTEXT ***
    # We store the data from the DB agent in the shared state.
    print(f"--- CONTEXT: Writing to tool_context.state['retrieved_data']: {db_agent_output[:50]}... ---")
    tool_context.state["retrieved_data"] = db_agent_output

    return db_agent_output


async def call_concierge_agent(question: str, tool_context: ToolContext):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("\n--- TOOL CALL: Delegating to concierge_agent ---")

    # *** READING FROM THE CONTEXT ***
    # We retrieve the data that the previous tool call saved.
    input_data = tool_context.state.get("retrieved_data", "No data found.")
    print(f"--- CONTEXT: Reading from tool_context.state['retrieved_data']: {input_data[:50]}... ---")


    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---
trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")


# --- Run the Multi-Agent System ---
await run_agent_query(
    trip_data_concierge_agent,
    "Find the top-rated hotels in San Francisco, then suggest a dinner spot near the one with the most reviews."
)

✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.

🚀 Running query for agent: 'trip_data_concierge'...



--- TOOL CALL: Delegating to db_agent ---
--- CONTEXT: Writing to tool_context.state['retrieved_data']: {'status': 'success', 'data': [{'name': 'The Grand... ---



--- TOOL CALL: Delegating to concierge_agent ---
--- CONTEXT: Reading from tool_context.state['retrieved_data']: {'status': 'success', 'data': [{'name': 'The Grand... ---



--------------------------------------------------
✅ Final Response:


The Seaside Inn has the most reviews.  A highly recommended dinner spot nearby is The Tide & Truffle, which has been described as having "occasionally well-meaning, yet utterly provincial, flourishes."


--------------------------------------------------



'The Seaside Inn has the most reviews.  A highly recommended dinner spot nearby is The Tide & Truffle, which has been described as having "occasionally well-meaning, yet utterly provincial, flourishes."\n'